# Diverse CF

In [ ]:
# | default_exp methods.dice

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# | export
from __future__ import annotations
from relax.import_essentials import *
from relax.methods.base import CFModule
from relax.base import BaseConfig
from relax.utils import auto_reshaping, grad_update, validate_configs

In [ ]:
#| hide
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from relax.ml_model import MLModule
from relax.data_module import load_data
from relax.ml_model import load_ml_module
import torch

## Util Functions



In [ ]:
# export
@jit
def dpp_style_vmap(cfs: Array):
    def dpp_fn(cf_1, cf_2):
        return 1 / (1 + jnp.linalg.norm(cf_1 - cf_2, ord=1))
    
    det_entries = vmap(vmap(dpp_fn, in_axes=(None, 0)), in_axes=(0, None))(cfs, cfs)
    det_entries += jnp.eye(cfs.shape[0]) * 1e-8
    assert det_entries.shape == (cfs.shape[0], cfs.shape[0])
    return jnp.linalg.det(det_entries)

In [ ]:
# From the original dice implementation
# https://github.com/interpretml/DiCE/blob/a772c8d4fcd88d1cab7f2e02b0bcc045dc0e2eab/dice_ml/explainer_interfaces/dice_pytorch.py#L222-L227
def dpp_style_torch(cfs: torch.Tensor):
    compute_dist = lambda x, y: torch.abs(x-y).sum()

    total_CFs = len(cfs)
    det_entries = torch.ones((total_CFs, total_CFs))
    for i in range(total_CFs):
        for j in range(total_CFs):
            det_entries[(i,j)] = 1.0/(1.0 + compute_dist(cfs[i], cfs[j]))
            if i == j:
                det_entries[(i,j)] += 1e-8
    return torch.det(det_entries)

In [ ]:
def jax2torch(x: Array):
    return torch.from_numpy(x.__array__())

In [ ]:
cfs = jrand.normal(jrand.PRNGKey(0), (100, 100))
cfs_tensor = jax2torch(cfs)
assert np.allclose(
    dpp_style_torch(cfs_tensor).numpy(),
    dpp_style_vmap(cfs)
)

Our jax-based implementation is ~500X faster than DiCE's pytorch implementation.

In [ ]:
%%timeit
torch_res = dpp_style_torch(cfs_tensor)

268 ms ± 4.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
jax_res = dpp_style_vmap(cfs)

501 µs ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
#| exporti
def _diverse_cf(
    x: jnp.DeviceArray,  # `x` shape: (k,), where `k` is the number of features
    y_target: Array, # `y_target` shape: (1,)
    pred_fn: Callable[[Array], Array],  # y = pred_fn(x)
    n_cfs: int,
    n_steps: int,
    lr: float,  # learning rate for each `cf` optimization step
    lambdas: Tuple[float, float, float, float], # (lambda_1, lambda_2, lambda_3, lambda_4)
    key: jrand.PRNGKey,
    validity_fn: Callable,
    cost_fn: Callable,
    apply_constraints_fn: Callable,
    compute_reg_loss_fn: Callable,
) -> Array:  # return `cf` shape: (k,)
    """Diverse Counterfactuals (Dice) algorithm."""

    def loss_fn(
        cfs: Array, # shape: (n_cfs, k)
        x: Array, # shape: (1, k)
        pred_fn: Callable[[Array], Array], # y = pred_fn(x)
        y_target: Array,
    ):
        cf_y_pred = pred_fn(cfs)
        loss_1 = validity_fn(y_target, cf_y_pred).mean()
        loss_2 = cost_fn(x, cfs).mean()
        loss_3 = - dpp_style_vmap(cfs).mean()
        loss_4 = compute_reg_loss_fn(x, cfs)
        return (
            lambda_1 * loss_1 + 
            lambda_2 * loss_2 + 
            lambda_3 * loss_3 + 
            lambda_4 * loss_4
        )
    
    @loop_tqdm(n_steps)
    def gen_cf_step(i, states: Tuple[Array, optax.OptState]):
        cf, opt_state = states
        grads = jax.grad(loss_fn)(cf, x, pred_fn, y_target)
        cf_updates, opt_state = grad_update(grads, cf, opt_state, opt)
        return cf, opt_state
    
    lambda_1, lambda_2, lambda_3, lambda_4 = lambdas
    key, subkey = jrand.split(key)
    cfs = jrand.normal(key, (n_cfs, x.shape[-1]))
    opt = optax.adam(lr)
    opt_state = opt.init(cfs)
    
    cfs, opt_state = lax.fori_loop(0, n_steps, gen_cf_step, (cfs, opt_state))
    # TODO: support return multiple cfs
    cfs = apply_constraints_fn(x, cfs[:1, :], hard=True)
    return cfs


## Config

In [ ]:
class DiverseCFConfig(BaseConfig):
    n_cfs: int = 5
    n_steps: int = 1000
    lr: float = 0.001
    lambda_1: float = 1.0
    lambda_2: float = 0.1
    lambda_3: float = 1.0
    lambda_4: float = 0.1
    validity_fn: str = 'KLDivergence'
    cost_fn: str = 'MeanSquaredError'
    seed: int = 42


In [ ]:
class DiverseCF(CFModule):

    def __init__(self, config: dict | DiverseCF = None, *, name: str = None):
        if config is None:
             config = DiverseCFConfig()
        config = validate_configs(config, DiverseCFConfig)
        name = "DiverseCF" if name is None else name
        super().__init__(config, name=name)

    @auto_reshaping('x')
    def generate_cf(
        self,
        x: Array,  # `x` shape: (k,), where `k` is the number of features
        pred_fn: Callable[[Array], Array],
        y_target: Array = None,
        rng_key: jnp.ndarray = None,
        **kwargs,
    ) -> jnp.DeviceArray:
        # TODO: Currently assumes binary classification.
        if y_target is None:
            y_target = 1 - pred_fn(x)
        else:
            y_target = jnp.array(y_target, copy=True)
        if rng_key is None:
            rng_key = jax.random.PRNGKey(self.config.seed)
        
        return _diverse_cf(
            x=x,  # `x` shape: (k,), where `k` is the number of features
            y_target=y_target,  # `y_target` shape: (1,)
            pred_fn=pred_fn,  # y = pred_fn(x)
            n_cfs=self.config.n_cfs,
            n_steps=self.config.n_steps,
            lr=self.config.lr,  # learning rate for each `cf` optimization step
            lambdas=(
                self.config.lambda_1, self.config.lambda_2, 
                self.config.lambda_3, self.config.lambda_4
            ),
            key=rng_key,
            validity_fn=keras.losses.get({'class_name': self.config.validity_fn, 'config': {'reduction': None}}),
            cost_fn=keras.losses.get({'class_name': self.config.cost_fn, 'config': {'reduction': None}}),
            apply_constraints_fn=self.apply_constraints_fn,
            compute_reg_loss_fn=self.compute_reg_loss_fn,
        )


In [ ]:
dm = load_data('dummy')
model = load_ml_module('dummy')
xs_train, ys_train = dm['train']
xs_test, ys_test = dm['test']

In [ ]:
dcf = DiverseCF()
dcf.init_fns()
cf = dcf.generate_cf(xs_test[0], model.pred_fn)
assert cf.shape == xs_test[0].shape

partial_gen = partial(dcf.generate_cf, pred_fn=model.pred_fn)
cfs = jax.vmap(partial_gen)(xs_test)

print("Validity: ", keras.metrics.binary_accuracy(
    (1 - model.pred_fn(xs_test)).round(),
    model.pred_fn(cfs)
).mean())

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Validity:  0.49600002
